### 예제
1. csv폴더에서 4개의 파일을 로드 (tran1, tran2, tran_d_1, trand_2)
2. tran_1과 tran_2는 단순한 행 추가 결합(유니언 결합)
3. tran_d_1과 trand_d_2도 유니언 결합
4. 2번,3번 과정에서 나온 데이터프레임을 특정 조건에 맞춰서 2번 데이터프레임 기준으로 열 추가 결합 (조인 결합)
5. csv 폴더에 있는 2개의 파일 로드(custom_master, item_master)
6. 4번 과정에서 나온 데이터프레임과 custom_master 조인 결합
7. 6번 과정에서 나온 데이터프레임과 item_master 조인결합
8. 결합이된 데이터프레임에서 quantity 컬럼과 item_price 컬럼의 데이터를 가지고 total_price라는 파생 변수를 생성 (total_price = item_price*quantitiy)

In [38]:
import os

In [39]:
import pandas as pd

In [40]:
tran_1 = pd.read_csv("../csv/tran_1.csv")
tran_2 = pd.read_csv("../csv/tran_2.csv")
tran_d_1 = pd.read_csv("../csv/tran_d_1.csv")
tran_d_2 = pd.read_csv("../csv/tran_d_2.csv")

In [41]:
result = pd.concat([tran_1, tran_2], axis = 0, ignore_index= True)
print(result)


     transaction_id   price         payment_date customer_id
0       T0000000113  210000  2019-02-01 01:36:57    PL563502
1       T0000000114   50000  2019-02-01 01:37:23    HD678019
2       T0000000115  120000  2019-02-01 02:34:19    HD298120
3       T0000000116  210000  2019-02-01 02:47:23    IK452215
4       T0000000117  170000  2019-02-01 04:33:46    PL542865
...             ...     ...                  ...         ...
6781    T0000006894  180000  2019-07-31 21:20:44    HI400734
6782    T0000006895   85000  2019-07-31 21:52:48    AS339451
6783    T0000006896  100000  2019-07-31 23:35:25    OA027325
6784    T0000006897   85000  2019-07-31 23:39:35    TS624738
6785    T0000006898   85000  2019-07-31 23:41:38    AS834214

[6786 rows x 4 columns]


In [42]:
result1 = pd.concat([tran_d_1, tran_d_2], axis = 0, ignore_index= True)
print(result1)


      detail_id transaction_id item_id  quantity
0             0    T0000000113    S005         1
1             1    T0000000114    S001         1
2             2    T0000000115    S003         1
3             3    T0000000116    S005         1
4             4    T0000000117    S002         2
...         ...            ...     ...       ...
7139       7139    T0000006894    S004         1
7140       7140    T0000006895    S002         1
7141       7141    T0000006896    S001         2
7142       7142    T0000006897    S002         1
7143       7143    T0000006898    S002         1

[7144 rows x 4 columns]


In [43]:
transcation_id = pd.merge(result, result1, on = 'transaction_id', how = "left")

In [44]:
customer = pd.read_csv("../csv/customer_master.csv")
item = pd.read_csv("../csv/item_master.csv")

In [45]:
total_df =pd.merge(transcation_id, customer, on = 'customer_id', how = "inner")

In [46]:
total_df = pd.merge(total_df, item, on = 'item_id', how = "inner")

In [52]:
total_df['total_price'] = total_df['item_price'] * total_df['quantity']


In [54]:
total_df.head(1)

,transaction_id,price,payment_date,customer_id,detail_id,item_id,quantity,name,class,gender,start_date,end_date,campaign_id,is_deleted,item_name,item_price,total_price
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,S005,1,XX,C01,M,2018-07-01 00:00:00,NaN,CA1,0,PC-E,210000,210000


In [55]:
group_data = total_df[['item_name','total_price']].groupby('item_name').sum()

In [56]:
# 내림차순 정렬
group_data.sort_values('total_price', ascending=False).index=[0]

ValueError: Length mismatch: Expected axis has 5 elements, new values have 1 elements

In [ ]:
final_r.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7144 entries, 0 to 7143
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  7144 non-null   object 
 1   price           7144 non-null   int64  
 2   payment_date    7144 non-null   object 
 3   customer_id     7144 non-null   object 
 4   detail_id       7144 non-null   int64  
 5   item_id         7144 non-null   object 
 6   quantity        7144 non-null   int64  
 7   name            6069 non-null   object 
 8   class           6069 non-null   object 
 9   gender          6069 non-null   object 
 10  start_date      6069 non-null   object 
 11  end_date        1978 non-null   object 
 12  campaign_id     6069 non-null   object 
 13  is_deleted      6069 non-null   float64
 14  item_name       7144 non-null   object 
 15  item_price      7144 non-null   int64  
 16  total_price     7144 non-null   int64  
dtypes: float64(1), int64(5), object(1

In [57]:
total_df['gender']

0       M
1       F
2       F
3       M
4       M
       ..
6064    M
6065    M
6066    M
6067    M
6068    M
Name: gender, Length: 6069, dtype: object

In [58]:
origin_data = total_df.copy()

In [59]:
# loc를 이용하여 m, f 를 변경
# m->남자, f->여자
total_df.loc[total_df['gender']  == "M", 'gender'] = '남자'
total_df.loc[total_df['gender']  == "F", 'gender'] = '여자'

In [60]:
total_df['gender'].value_counts()

남자    3268
여자    2801
Name: gender, dtype: int64

In [61]:
total_df = origin_data.copy()

In [62]:
total_df['gender'].value_counts()

M    3268
F    2801
Name: gender, dtype: int64

In [63]:
# apply() 사용하여 남자, 여자 변경

def change(x):
    # x 에는 M아니면 F
    if x == "F":
        result = "여자"
    else:
        result = "남자"
    return result

total_df['gender'].apply(change)

0       남자
1       여자
2       여자
3       남자
4       남자
        ..
6064    남자
6065    남자
6066    남자
6067    남자
6068    남자
Name: gender, Length: 6069, dtype: object

In [68]:
total_df['gender'].apply(lambda x : "남자" if (x == 'M') else ("여자" if(x == "F") else "무응답"))

0       남자
1       여자
2       여자
3       남자
4       남자
        ..
6064    남자
6065    남자
6066    남자
6067    남자
6068    남자
Name: gender, Length: 6069, dtype: object

In [69]:
# payment_date 컬럼을 시계열 데이터 변경
# to_datetime()
total_df.head(1)

,transaction_id,price,payment_date,customer_id,detail_id,item_id,quantity,name,class,gender,start_date,end_date,campaign_id,is_deleted,item_name,item_price,total_price
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,S005,1,XX,C01,M,2018-07-01 00:00:00,NaN,CA1,0,PC-E,210000,210000


In [71]:
total_df['payment_date'] = pd.to_datetime(total_df['payment_date'], format='%Y-%m-%d %H:%M:%S')

In [72]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6069 entries, 0 to 6068
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   transaction_id  6069 non-null   object        
 1   price           6069 non-null   int64         
 2   payment_date    6069 non-null   datetime64[ns]
 3   customer_id     6069 non-null   object        
 4   detail_id       6069 non-null   int64         
 5   item_id         6069 non-null   object        
 6   quantity        6069 non-null   int64         
 7   name            6069 non-null   object        
 8   class           6069 non-null   object        
 9   gender          6069 non-null   object        
 10  start_date      6069 non-null   object        
 11  end_date        1978 non-null   object        
 12  campaign_id     6069 non-null   object        
 13  is_deleted      6069 non-null   int64         
 14  item_name       6069 non-null   object        
 15  item

In [73]:
# 시계열 데이터에서 월-년 추출해서 새로운 파생변수 payment_month 생성
total_df['payment_month'] = total_df['payment_date'].dt.strftime('%m-%Y')

In [74]:
total_df.head(1)

,transaction_id,price,payment_date,customer_id,detail_id,item_id,quantity,name,class,gender,start_date,end_date,campaign_id,is_deleted,item_name,item_price,total_price,payment_month
0,T0000000113,210000,2019-02-01 01:36:57,PL563502,0,S005,1,XX,C01,M,2018-07-01 00:00:00,NaN,CA1,0,PC-E,210000,210000,02-2019


In [76]:
# 월별 거래량을 확인
group_data = total_df[['payment_month', 'total_price']].groupby('payment_month').agg(['mean', 'sum'])

In [77]:
group_data

total_price           
                        mean        sum
payment_month                          
02-2019        137592.964824  136905000
03-2019        135154.382470  135695000
04-2019        136106.106106  135970000
05-2019        133551.307847  132750000
06-2019        135747.800587  138870000
07-2019        138377.609108  145850000

In [ ]:
!pip install matplotlib

  Using cached matplotlib-3.7.1-cp311-cp311-win_amd64.whl (7.6 MB)
  Using cached contourpy-1.0.7-cp311-cp311-win_amd64.whl (162 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [78]:
pivot_data = pd.pivot_table(
    total_df, 
    index = 'payment_month', 
    aggfunc = ['sum', 'mean', 'min', 'max', 'std'], 
    values = 'total_price'
)